In [ ]:
#|default_exp callback.utils

# Callback Utilities
> Additional functionality for fastai callbacks

In [ ]:
#|export
from __future__ import annotations

from fastai.callback.schedule import SchedCos, _Annealer

from fastxtend.imports import *

In [ ]:
#|hide
from nbdev.showdoc import show_doc

## Callback Scheduler

In [ ]:
#|exporti
class CallbackScheduler():
    "A mixin for scheduling values in a Callback"

    def setup_schedule(self,
        n_epoch:int, # Number of training epochs. From a callback pass n_epoch
        dls_len:int, # Length of the training dataset
        start_value:Numeric, # Initial scheduling value
        final_value:Numeric, # Final scheduling value
        start:Numeric, # Start schedule in percent of training steps (float) or epochs (int, index 0)
        finish:Numeric, # Finish schedule in percent of training steps (float) or epochs (int, index 0)
        schedule:Callable[..., _Annealer]=SchedCos, # Schedule type. Any fastai schedule annealer
        ndigits:Numeric|None=None, # Round return value to ndigits if set using Python's `round`.
        callback_name:str="CallbackScheduler", # Name of scheduler for warning & error messages
        resume:bool=False, # Whether training has resumed or not
    ):
        "Setup the schedule for `schedule_step`. Call during `Callback.begin_fit`"
        if start > finish:
            raise ValueError(f'{callback_name} schedule {start=} is greater then {finish=}')

        if finish >= 1 and isinstance(finish, int):
            finish = finish/n_epoch
        if finish > 1:
            warn(f'{callback_name} {finish=} is  greater than one and will not finish in this training run')

        if resume and n_epoch < finish*n_epoch:
            raise ValueError(f"Resuming {callback_name} before the schedule is finished is not supported")

        self.final_value = final_value
        self.ndigits = ndigits
        self.schedule_pct = 0.
        self.schedule = schedule(start_value, final_value)
        self._schedule_sched = 1/(dls_len * n_epoch * (finish - start))

    def schedule_step(self,
        value:Numeric, # Value to schedule. Value is returned as the passed in type
        pct_train:float # Training progress in percent. From a callback pass self.pct_train
    ):
        "Takes a scheduling step and returns updated value"
        value_type = type(value)
        if pct_train >= self.start and value != self.final_value:
            if pct_train >= self.finish:
                value = self.final_value
            else:
                value = self.schedule(self.schedule_pct)
                self.schedule_pct += self._schedule_sched
        if self.ndigits is not None:
            return value_type(round(value, self.ndigits))
        else:
            return value_type(value)

In [ ]:
show_doc(CallbackScheduler)

In [ ]:
show_doc(CallbackScheduler.setup_schedule)

In [ ]:
show_doc(CallbackScheduler.schedule_step)